In [1]:
import os 
import json
import csv
import xml.etree.ElementTree as ET

from random import shuffle

Create directory to save human evaluation sheets

In [2]:
if not os.path.exists('human'):
    os.mkdir('human')

# WebNLG

In [1]:
# find testset of human evaluation in the first challenge

path = os.path.join('results', 'webnlg_t5', 'result.json')
data = json.load(open(path))

with open('human/webnlg/sample_ids.txt') as f:
    sample_ids = f.read().split()

tree = ET.parse('human/webnlg/testnolex.xml')
root = tree.getroot().find('entries')

idxs = []
for entry in root:
    category = entry.attrib['category']
    eid = entry.attrib['eid'].replace('Id', '')
    if eid in sample_ids:
        modifiedtripleset = []
        for triple in entry.find('modifiedtripleset').findall('mtriple'):
            subj, pred, obj = triple.text.split(' | ')
            modifiedtripleset.append({
                'subject': subj,
                'property': pred,
                'object': obj
            })
        triples = []
        for triple in sorted(modifiedtripleset, key=lambda x: (x['property'], x['subject'], x['object'])):
            subj, pred, obj = triple['subject'], triple['property'], triple['object']
            str_triple = ' '.join(['<subject>', subj, '<predicate>', pred, '<object>', obj])
            triples.append(str_triple)
        triples = ' <triple> '.join(triples)
        inp = ' '.join([category, triples])
        
        for i, row in enumerate(data):
            if row['intent'] == inp:
                idxs.append(i)
                
path = os.path.join('human/webnlg', 'idxs.json')
json.dump(idxs, open(path, 'w'), separators=(',', ':'), sort_keys=True, indent=4)

In [5]:
output = []
for fdir in os.listdir('results'):
    if 'webnlg' in fdir:
        path = os.path.join('results', fdir, 'result.json')
        result = json.load(open(path))
        
        for idx in idxs:
            f = data[idx]
            output.append({
                'idx': idx,
                'intent': ' <triple>\n '.join(f['intent'].split(' <triple> ')),
                'hyp': f['hyp'].strip(),
                'ref': f['refs'][0],
                'domain': fdir
            })
            
for idx in idxs:
    f = data[idx]
    output.append({
        'idx': idx,
        'intent': ' <triple>\n '.join(f['intent'].split(' <triple> ')),
        'hyp': f['refs'][0],
        'ref': f['refs'][0],
        'domain': 'gold'
    })

shuffle(output)
# save idxs
fdir = os.path.join('human', 'webnlg')
path = os.path.join(fdir, 'result.json')
json.dump(output, open(path, 'w'), separators=(',', ':'), sort_keys=True, indent=4)

In [7]:
fdir = os.path.join('human', 'webnlg')
path = os.path.join(fdir, 'annotation.csv')

header = ["MR", "Reference", "Hypothesis", \
          "Incorrect Number", "Incorrect Named Entity", \
          "Incorrect Word", "Context Error", "Not checkable", "Other", "Fluency"]

with open(path, 'w') as f:
    writer = csv.writer(f, delimiter=';', quotechar='\"', quoting=csv.QUOTE_MINIMAL)
    writer.writerow(header)
    
    for row in output:
        writer.writerow([row['intent'], row['ref'], row['hyp']])